In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/breast-cancer-dataset/Y_train.npy
/kaggle/input/breast-cancer-dataset/X_test.npy
/kaggle/input/breast-cancer-dataset/X_train.npy


In [3]:
import numpy as np
import tensorflow as tf

# Load the data
X_train = np.load('/kaggle/input/breast-cancer-dataset/X_train.npy')
Y_train = np.load('/kaggle/input/breast-cancer-dataset/Y_train.npy')
X_test = np.load('/kaggle/input/breast-cancer-dataset/X_test.npy')

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Reshape Y_train
Y_train = Y_train.flatten()

# Split the test dataset into test and validation sets
test_split_ratio = 0.5
split_index = int(len(X_test) * test_split_ratio)

X_test_split = X_test[:split_index]
X_val_split = X_test[split_index:]

# Save the split test and validation datasets as .npy files
np.save('X_test_split.npy', X_test_split)
np.save('X_val_split.npy', X_val_split)

# Print the shapes of the split datasets
print("X_test_split shape:", X_test_split.shape)
print("X_val_split shape:", X_val_split.shape)


2024-04-20 13:21:24.580624: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 13:21:24.580720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 13:21:24.689341: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


X_test_split shape: (693, 50, 50, 3)
X_val_split shape: (694, 50, 50, 3)


In [4]:
# Confirm the shapes
print("Shapes of X_train and Y_train:", X_train.shape, Y_train.shape)
print("Shapes of X_test and Y_test:", X_test.shape, Y_test.shape)

Shapes of X_train and Y_train: (4160, 50, 50, 3) (4160,)


NameError: name 'Y_test' is not defined

In [5]:
import numpy as np
import tensorflow as tf

class NumpyDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, y, batch_size=32, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return len(self.X) // self.batch_size
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_X = self.X[indexes]
        batch_y = self.y[indexes]
        return batch_X, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


# Load the data
X_train = np.load('/kaggle/input/breast-cancer-dataset/X_train.npy')
Y_train = np.load('/kaggle/input/breast-cancer-dataset/Y_train.npy')

# Load the split test and validation datasets
X_test = np.load('/kaggle/working/X_test_split.npy')
Y_test = np.load('/kaggle/working/X_val_split.npy')            
# Load the data
#X_train = np.load('/kaggle/input/breast-cancer-dataset/X_train.npy')
#Y_train = np.load('/kaggle/input/breast-cancer-dataset/Y_train.npy')
#X_test = np.load('/kaggle/input/breast-cancer-dataset/X_test.npy')
#Y_test = np.load('/kaggle/input/breast-cancer-dataset/X_test.npy')

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Reshape Y_train and Y_test
Y_train = Y_train.flatten()
Y_test = Y_test.flatten()

# Define batch size
batch_size = 32

# Create data generators
train_generator = NumpyDataGenerator(X_train, Y_train, batch_size=batch_size)
test_generator = NumpyDataGenerator(X_test, Y_test, batch_size=batch_size)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-04-20 13:22:08.391971: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 511: 3.99746, expected 3.4575
2024-04-20 13:22:08.392064: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4608: 6.74587, expected 5.87283
2024-04-20 13:22:08.392075: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4609: 6.53509, expected 5.66205
2024-04-20 13:22:08.392083: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4610: 6.96058, expected 6.08753
2024-04-20 13:22:08.392092: E external/local_xla/xla/service/gpu/buffer_compar

 49/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5080 - loss: 0.7255

I0000 00:00:1713619330.243799      82 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


130/130 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.5098 - loss: 0.7088 - val_accuracy: 0.0000e+00 - val_loss: 0.6870
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5097 - loss: 0.6931 - val_accuracy: 0.0000e+00 - val_loss: 0.6868
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4942 - loss: 0.6933 - val_accuracy: 0.0000e+00 - val_loss: 0.6872
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5094 - loss: 0.6931 - val_accuracy: 0.0000e+00 - val_loss: 0.6881
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5040 - loss: 0.6931 - val_accuracy: 0.0000e+00 - val_loss: 0.6895
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.0000e+00 - val_loss: 0.6889
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4986 - loss: 0.6932 - val_accuracy: 0.0000e+00 - val_loss: 0.6878
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4912 - loss: 0.6933 -

In [8]:
import numpy as np

# Convert probabilities to binary labels
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Evaluate the model
from sklearn.metrics import classification_report

true_labels = Y_test[:len(predictions)].astype(int)  # Ensure true labels have the same length as predictions
print(classification_report(true_labels, predicted_labels))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00     672.0
           1       0.00      0.00      0.00       0.0

    accuracy                           0.00     672.0
   macro avg       0.00      0.00      0.00     672.0
weighted avg       0.00      0.00      0.00     672.0



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.

In [9]:
import numpy as np
import tensorflow as tf

class NumpyDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, y, batch_size=32, shuffle=True, augment=False):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment  # Flag for data augmentation
        self.indexes = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return len(self.X) // self.batch_size
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_X = self.X[indexes]
        batch_y = self.y[indexes]
        if self.augment:
            batch_X = self.apply_augmentation(batch_X)
        return batch_X, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def apply_augmentation(self, batch_X):
        # Apply data augmentation techniques
        # Example: random horizontal flip
        batch_X = tf.image.random_flip_left_right(batch_X)
        return batch_X
    
#     def apply_augmentation(self, batch_X):
#         # Apply data augmentation techniques
#         batch_X = tf.image.random_flip_left_right(batch_X)
#     # Randomly rotate the image by 90-degree increments
#         batch_X = tf.image.rot90(batch_X, tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
#     # Randomly zoom into the image by a factor between 0.8 and 1.2
#         height, width = tf.shape(batch_X)[1], tf.shape(batch_X)[2]
#         new_height = tf.random.uniform(shape=[], minval=int(0.8 * tf.cast(height, tf.float32)), maxval=int(1.2 * tf.cast(height, tf.float32)), dtype=tf.int32)
#         new_width = tf.random.uniform(shape=[], minval=int(0.8 * tf.cast(width, tf.float32)), maxval=int(1.2 * tf.cast(width, tf.float32)), dtype=tf.int32)
#         batch_X = tf.image.resize(batch_X, (new_height, new_width))
#         return batch_X



# Load the data
X_train = np.load('/kaggle/input/breast-cancer-dataset/X_train.npy')
Y_train = np.load('/kaggle/input/breast-cancer-dataset/Y_train.npy')

# Load the split test and validation datasets
X_test = np.load('/kaggle/working/X_test_split.npy')
Y_test = np.load('/kaggle/working/X_val_split.npy')            

# Normalize pixel values
X_train = X_train / 255.0
X_test = X_test / 255.0

# Reshape Y_train and Y_test
Y_train = Y_train.flatten()
Y_test = Y_test.flatten()

# Define batch size
batch_size = 32

# Create data generators with augmentation enabled for training data
train_generator = NumpyDataGenerator(X_train, Y_train, batch_size=batch_size, augment=True)
test_generator = NumpyDataGenerator(X_test, Y_test, batch_size=batch_size)

# Define the model with dropout regularization
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),  # Add dropout layer with dropout rate of 0.5
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.5804 - loss: 0.6783 - val_accuracy: 0.0000e+00 - val_loss: 0.6168
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6877 - loss: 0.6077 - val_accuracy: 0.0000e+00 - val_loss: 0.6522
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7132 - loss: 0.5765 - val_accuracy: 0.0000e+00 - val_loss: 0.8259
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7310 - loss: 0.5537 - val_accuracy: 0.0000e+00 - val_loss: 0.9405
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7039 - loss: 0.5810 - val_accuracy: 0.0000e+00 - val_loss: 0.8354
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7279 - loss: 0.5694 - val_accuracy: 0.0000e+00 - val_loss: 0.7672
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7502 - loss: 0.5300 - val_accuracy: 0.0000e+00 - val_loss: 4.4206
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7410 - loss: 0.5383 -

In [10]:
import numpy as np
import tensorflow as tf

class NumpyDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, X, y, batch_size=32, shuffle=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.X))
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return len(self.X) // self.batch_size
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_X = self.X[indexes]
        batch_y = self.y[indexes]
        return batch_X, batch_y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


# Load the data
X_train = np.load('/kaggle/input/breast-cancer-dataset/X_train.npy')
Y_train = np.load('/kaggle/input/breast-cancer-dataset/Y_train.npy')

# Normalize pixel values
X_train = X_train / 255.0

# Reshape Y_train
Y_train = Y_train.flatten()

# Define batch size
batch_size = 32

# Create data generator
train_generator = NumpyDataGenerator(X_train, Y_train, batch_size=batch_size)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10)



Epoch 1/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5337 - loss: 0.6930
Epoch 2/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6831 - loss: 0.6021
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7257 - loss: 0.5649
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7328 - loss: 0.5573
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7314 - loss: 0.5385
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7622 - loss: 0.5151
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7370 - loss: 0.5407
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7389 - loss: 0.5362
Epoch 9/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7462 - loss: 0.5201
Epoch 10/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7569 - loss: 0.5242


In [11]:
import numpy as np

# Load the single test data file
X_test_single = np.load('/kaggle/input/breast-cancer-dataset/X_test.npy')

# Normalize pixel values
X_test_single = X_test_single / 255.0

# Reshape the data if necessary
# X_test_single = np.reshape(X_test_single, (num_samples, width, height, num_channels))

# Make predictions using the trained model
predictions = model.predict(X_test_single)

# If your model predicts probabilities, you may want to convert them to class labels
# For example, if your model is trained for binary classification with sigmoid activation
class_labels = (predictions > 0.5).astype(int)

# Print the predictions
print(class_labels)


33/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step   

2024-04-20 13:29:05.575517: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4611: 8.58576, expected 7.6197
2024-04-20 13:29:05.575571: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4613: 8.21984, expected 7.25378
2024-04-20 13:29:05.575580: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4615: 7.96022, expected 6.99417
2024-04-20 13:29:05.575589: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4617: 8.55713, expected 7.59108
2024-04-20 13:29:05.575597: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4618: 8.62198, expected 7.65593
2024-04-20 13:29:05.575605: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4619: 7.93489, expected 6.96883
2024-04-20 13:29:05.575613: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4620: 8.58683, expected 7.62077
2024-04-20 13:29:05.575630: E exter

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
[[0]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]


In [12]:
# Convert probabilities to binary class labels
class_labels = (predictions > 0.5).astype(int).flatten()

# Count the occurrences of each class label
num_benign = np.sum(class_labels == 0)
num_malignant = np.sum(class_labels == 1)

print("Number of benign instances:", num_benign)
print("Number of malignant instances:", num_malignant)


Number of benign instances: 655
Number of malignant instances: 732


In [13]:
import pandas as pd

# Convert probabilities to binary class labels
class_labels = (predictions > 0.5).astype(int).flatten()

# Create a DataFrame with the test dataset and predicted labels
test_df = pd.DataFrame({'ImageId': np.arange(len(class_labels)) + 1, 'Label': class_labels})

# Separate into benign and malignant DataFrames
benign_df = test_df[test_df['Label'] == 0]
malignant_df = test_df[test_df['Label'] == 1]

# Save to CSV files
benign_df.to_csv('/kaggle/working/benign_predictions.csv', index=False)
malignant_df.to_csv('/kaggle/working/malignant_predictions.csv', index=False)

print("CSV files saved successfully.")


CSV files saved successfully.
